# Text Classification to detect sensitive data exposure

## Importing libraries and downloading the dataset

In [ ]:

import tensorflow as tf
import json
import os
import random
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


git_folder = "/content/Detection-of-Sensitive-Data-Exposure-in-Images"
if os.path.exists(git_folder) == False:
  !git clone https://github.com/amjj6/Detection-of-Sensitive-Data-Exposure-in-Images

dataset_folder = git_folder + "/text_dataset/"
sensitive_datafile = "SensitiveDataset.json"
nonsensitive_datafile = "NonSensitiveDataset.json"

Cloning into 'Detection-of-Sensitive-Data-Exposure-in-Images'...
remote: Enumerating objects: 1070, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 1070 (delta 4), reused 10 (delta 2), pack-reused 1057
Receiving objects: 100% (1070/1070), 236.73 MiB | 31.34 MiB/s, done.
Resolving deltas: 100% (133/133), done.
Checking out files: 100% (741/741), done.


In [ ]:
np.random.seed(500)

In [ ]:

vocab_size = 3000
embedding_dim = 32
max_length = 60
truncation_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [ ]:
Corpus1 = pd.read_json(r"/content/Detection-of-Sensitive-Data-Exposure-in-Images/text_dataset/NonSensitiveDataset.json",encoding='latin-1')
Corpus2 = pd.read_json(r"/content/Detection-of-Sensitive-Data-Exposure-in-Images/text_dataset/SensitiveDataset.json",encoding='latin-1')
Corpus= pd.concat([Corpus1, Corpus2])
print (Corpus.shape)
print (Corpus1.shape)
print (Corpus2.shape)
print(np.unique(Corpus1['is_sensitive']))
print(np.unique(Corpus2['is_sensitive']))
print(np.unique(Corpus['is_sensitive']))

(31576, 2)
(16000, 2)
(15576, 2)
[0]
[1]
[0 1]


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
for entry in Corpus['data']:
  print(entry)
  print(type(entry))
  print(entry.lower())
  break
print(np.unique(Corpus['is_sensitive']))

A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]
<class 'str'>
a bartender is working at a saloon, serving drinks to customers. after he fills a stereotypically irish man's bucket with beer, carrie nation and her followers burst inside. they assault the irish man, pulling his hat over his eyes and then dumping the beer over his head. the group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. the bartender then sprays seltzer water in nation's face before a group of policemen appear and order everybo

In [ ]:

Corpus['data'].dropna(inplace=True)

Corpus['data'] = [entry.lower() for entry in Corpus['data']]
print(np.unique(Corpus['is_sensitive']))

[0 1]


In [ ]:
nltk.download('averaged_perceptron_tagger')

Corpus['data']= [word_tokenize(entry) for entry in Corpus['data']]

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

print(np.unique(Corpus['is_sensitive']))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[0 1]


In [ ]:
nltk.download('stopwords')
print(Corpus.shape)
print(np.unique(Corpus['is_sensitive']))
print(Corpus[:2000].shape)
from sklearn.utils import shuffle
Corpus = shuffle(Corpus)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(31576, 2)
[0 1]
(2000, 2)


In [ ]:
Corpus = Corpus[:2000]
np.unique(Corpus['is_sensitive'])

array([0, 1])

In [ ]:
Corpus = Corpus[~Corpus.index.duplicated()]
print(np.unique(Corpus['is_sensitive']))

[0 1]


In [ ]:

for index,entry in enumerate(Corpus['data']):
    print(index)
    
    Final_words = []
    
    word_Lemmatized = WordNetLemmatizer()
    
    for word, tag in pos_tag(entry):
        
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    
    Corpus.loc[index,'text_final'] = str(Final_words)

0
1
2
3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
27

## Preprocessing the dataset

In [ ]:
print(Corpus.columns)
Corpus = Corpus[Corpus['is_sensitive'].notna()]
Corpus = Corpus[Corpus['text_final'].notna()]
print(np.unique(Corpus['is_sensitive']))

Index(['is_sensitive', 'data', 'text_final'], dtype='object')
[0. 1.]


In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['is_sensitive'],test_size=0.3)

np.unique(Train_Y)
print(Train_X.head(10))


546     ['feud', 'max', 'walter', 'matthau', 'john', '...
1312    ['baywatch', 'lifeguard', 'mitch', 'stephanie'...
1339    ['username', 'minette', 'password', 'ricardo',...
59      ['world', 'war', 'ii', 'fly', 'tiger', 'triple...
310     ['username', 'toby', 'password', 'conan', 'ema...
853     ['username', 'dona', 'password', 'carmela', 'e...
343     ['troop', 'massacre', 'furnace', 'creek', 'for...
301     ['two', 'college', 'student', 'keith', 'aj', '...
1318    ['four', 'friend', 'bob', 'bailey', 'tone', 't...
562     ['username', 'ambrose', 'password', 'steaua', ...
Name: text_final, dtype: object


## Model

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
print(Tfidf_vect.vocabulary_)

{'western': 4876, 'front': 2073, 'world': 4946, 'war': 4827, 'american': 201, 'battalion': 479, 'advance': 79, 'french': 2067, 'town': 4527, 'nevremont': 2912, 'sergeant': 3860, 'bill': 564, 'thatcher': 4439, 'boyd': 652, 'leave': 2531, 'charge': 891, 'small': 4027, 'rear': 3420, 'guard': 2165, 'four': 2050, 'men': 2774, 'cover': 1230, 'retreat': 3578, 'cut': 1300, 'heavy': 2220, 'shelling': 3910, 'try': 4603, 'comfort': 1058, 'wound': 4953, 'first': 2009, 'bud': 714, 'russell': 3703, 'gleason': 2124, 'relate': 3490, 'story': 4195, 'join': 2433, 'military': 2816, 'family': 1946, 'farm': 1956, 'enlist': 1770, 'despite': 1447, 'mother': 2857, 'plea': 3135, 'become': 500, 'involved': 2384, 'foreign': 2041, 'conflict': 1111, 'finish': 2006, 'die': 1485, 'second': 3817, 'doughboy': 1596, 'lew': 2550, 'cavanaugh': 843, 'cody': 1020, 'new': 2913, 'york': 4987, 'playboy': 3130, 'use': 4719, 'enlistment': 1771, 'way': 4848, 'final': 2000, 'night': 2921, 'pleasure': 3138, 'one': 2956, 'conquest'

In [ ]:
print(Train_X_Tfidf)

  (0, 4948)	0.09774084904133441
  (0, 4899)	0.02842337054507777
  (0, 4863)	0.044996447588495456
  (0, 4848)	0.0302784152977474
  (0, 4820)	0.048870424520667206
  (0, 4711)	0.03690102260573007
  (0, 4486)	0.02842337054507777
  (0, 4452)	0.07248367385333361
  (0, 4407)	0.02617961851709364
  (0, 4362)	0.02283869470316289
  (0, 4340)	0.04673842116682122
  (0, 4269)	0.04673842116682122
  (0, 4252)	0.044996447588495456
  (0, 4215)	0.11098606365729975
  (0, 4189)	0.0313612031970099
  (0, 4119)	0.048870424520667206
  (0, 4102)	0.055493031828649876
  (0, 4076)	0.051619054896478125
  (0, 3994)	0.048870424520667206
  (0, 3934)	0.14661127356200163
  (0, 3897)	0.04224781721268454
  (0, 3869)	0.031751232972530116
  (0, 3829)	0.027360676034337734
  (0, 3813)	0.04673842116682122
  (0, 3714)	0.04673842116682122
  :	:
  (169, 2695)	0.16094225267777332
  (169, 2692)	0.09141233354732871
  (169, 2619)	0.14239207428230655
  (169, 2593)	0.07858434383253916
  (169, 2458)	0.16721019722560368
  (169, 2427)	0.2

In [ ]:

Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)

predictions_NB = Naive.predict(Test_X_Tfidf)

print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  51.35135135135135


In [ ]:

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

predictions_SVM = SVM.predict(Test_X_Tfidf)

print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  52.702702702702695


In [ ]:
np.unique(Train_Y)

array([0])